# Imports & Setup

In [11]:
!pip install -q google-cloud-storage==1.43.0
from pathlib import Path
import pickle
from collections import Counter, OrderedDict, defaultdict
# !pip install -q graphframes
# import pyspark
# import sys
# import itertools
# from itertools import islice, count, groupby
# import pandas as pd
# import os
# import re
# from operator import itemgetter
# import nltk
# from nltk.stem.porter import *
# from nltk.corpus import stopwords
# from time import time
# from pathlib import Path

# import pandas as pd
# from google.cloud import storage

# import hashlib
# def _hash(s):
#     return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

# nltk.download('stopwords')
# from pyspark.sql import *
# from pyspark.sql.functions import *
# from pyspark import SparkContext, SparkConf, SparkFiles
# from pyspark.sql import SQLContext
# from graphframes import *

# Creating Page views

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp


In [12]:
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# read in the counter
# with open(pv_clean, 'rb') as f:
#   wid2pv = pickle.loads(f.read())

In [15]:
print(pv_clean)

pageviews-202108-user.pkl


In [16]:
bucket_name = 'ir_assg3_eithan' 
dict_name = "page_views_dict.pkl"
dict_dst = f'gs://{bucket_name}/indexes/{dict_name}'
!gsutil cp $pv_clean $dict_dst

!gsutil ls -lh $dict_dst

Copying file://pageviews-202108-user.pkl [Content-Type=application/octet-stream]...
- [1 files][ 73.5 MiB/ 73.5 MiB]                                                
Operation completed over 1 objects/73.5 MiB.                                     
  73.5 MiB  2023-01-05T22:36:09Z  gs://ir_assg3_eithan/indexes/page_views_dict.pkl
TOTAL: 1 objects, 77070900 bytes (73.5 MiB)
